In [124]:
# '../data/raw/trainstation_sa2_perhouse.csv'

import os
import json
import pandas as pd
from typing import Dict, Any
from urllib.request import urlretrieve
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from shapely.geometry import Point
from shapely import wkt
import openrouteservice
import polars as pl
import os

## Extra preprocessing and Joining data sets 

In [125]:
df = pd.read_csv('../data/raw/trainstation_sa2_perhouse.csv')

# 

In [126]:
df_copy = df.copy()

In [127]:
df_rf = pd.read_parquet('../data/landing/domain_data.parquet')


In [128]:
df_rf

,agency_id,listing_id,limited_agency_mode,promotion_level,photo_count,property_type,num_bedrooms,num_bathrooms,parking,is_rural,...,actual_listing_date,num_days_listed,inspections_count,last_modification_date,postcode,latitude,longitude,has_photo,has_floorplan,num_visible_schools
0,24974,10011763,False,ELITE,8,Apartment / Unit / Flat,1,1,1,False,...,2024-08-20T11:38:40.000,16,0,2024-08-20T11:38:40.08,3182,-37.860843,144.983995,True,True,3
1,24974,10026864,False,ELITE,13,Apartment / Unit / Flat,2,2,0,False,...,2024-08-19T12:28:26.000,17,0,2024-08-19T12:28:25.607,3000,-37.809309,144.959739,True,False,3
2,11740,10073916,False,STANDARD,8,Apartment / Unit / Flat,3,2,1,False,...,2024-08-28T11:34:02.000,8,0,2024-08-28T11:34:01.863,3033,-37.747722,144.866798,True,False,3
3,24974,10119117,False,ELITE,6,Apartment / Unit / Flat,1,1,1,False,...,2024-07-23T11:50:42.000,44,0,2024-08-12T12:13:52.853,3016,-37.859629,144.890406,True,False,3
4,21959,10128494,False,STANDARD,9,House,4,2,1,False,...,2024-08-09T10:12:55.000,27,0,2024-08-14T10:08:26.203,3167,-37.937272,145.093406,True,False,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16002,24974,9940435,False,ELITE,6,House,3,2,2,False,...,2024-08-19T11:41:05.000,17,0,2024-08-19T11:41:04.73,3338,-37.717389,144.597336,True,False,3
16003,24974,9945996,False,ELITE,7,Apartment / Unit / Flat,2,1,1,False,...,2024-08-27T12:00:54.000,9,0,2024-08-27T12:00:53.523,3144,-37.870108,145.035458,True,True,3
16004,20612,9948218,False,STANDARD,15,House,4,2,2,False,...,2024-08-28T22:47:22.000,7,0,2024-08-28T22:48:26.907,3029,-37.834144,144.684018,True,False,3
16005,24974,9960759,False,ELITE,4,House,3,1,0,False,...,2024-08-21T14:57:16.000,15,0,2024-08-21T14:57:15.53,3181,-37.854055,145.004678,True,True,3


In [129]:
df.drop(columns=['postcode'], inplace=True)

In [130]:
df['Listing date'] = pd.to_datetime(df['Listing date'])
df['Year'] = df['Listing date'].dt.year
df['Month'] = df['Listing date'].dt.month

#  early pre-processing

/var/folders/lh/3pyg7bbj38b6_468dvhq3hnm0000gp/T/ipykernel_18356/4113306267.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Listing date'] = pd.to_datetime(df['Listing date'])


In [131]:
df = df.drop(columns=['Listing date'])

In [132]:
df['Year'].value_counts()

Year
2015    165721
2016    155504
2017    149742
2013    128085
2014    114351
2012     32186
Name: count, dtype: int64

In [133]:
# df = df.drop('STOP_NAME', axis=1)

In [134]:
df.rename(columns={'distance': 'nearst_train_station_dist'}, inplace=True)

changed the column names to fit both the data toghether...
further group some similar houeses 

In [135]:
print(list(df['property_type'].unique()))
df['property_type'] = df['property_type'].replace({'unit':'Apartment / Unit / Flat','flat':'Apartment / Unit / Flat','apartment':'Apartment / Unit / Flat'})

['townhouse', 'house', 'apartment', 'unit', 'villa', 'studio', 'flat', 'duplex-semi-detached', 'duplex', 'serviced apartment', 'retirement living']


In [136]:
print(list(df['property_type'].unique()))
new_domaindf  = pd.read_csv('../data/curated/domain.csv')

['townhouse', 'house', 'Apartment / Unit / Flat', 'villa', 'studio', 'duplex-semi-detached', 'duplex', 'serviced apartment', 'retirement living']


In [137]:
new_domaindf

,property_type,num_bedrooms,num_bathrooms,parking,rent,suburb,postcode,latitude,longitude,month,year,distance_to_cbd,nearst_train_station_dist
0,house,3,1,2,420.0,sunshine north,3020,-37.774926,144.834817,2,2024,11.923216,0.940065
1,house,3,2,0,850.0,coburg,3058,-37.748181,144.958473,9,2024,6.901368,0.758250
2,house,4,2,2,1300.0,camberwell,3124,-37.842214,145.071069,8,2024,10.182372,0.441276
3,house,4,2,2,580.0,tarneit,3029,-37.835021,144.669242,6,2024,25.979382,6.039059
4,house,4,2,2,440.0,winter valley,3358,-37.567505,143.784207,8,2024,107.405017,82.924725
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14693,townhouse,4,3,2,1050.0,burwood east,3151,-37.856688,145.132631,9,2023,15.825365,2.098144
14694,house,4,2,2,560.0,werribee south,3030,-37.963369,144.707632,9,2022,28.143669,8.197645
14695,apartment / unit / flat,3,2,2,700.0,ferntree gully,3156,-37.882581,145.267643,9,2022,28.016795,2.431007
14696,townhouse,3,3,2,835.0,coburg,3058,-37.737269,144.982527,8,2024,8.289849,1.594189


In [138]:
df.drop(columns=['unique_id'], inplace=True)
# 

In [139]:
# make the columns lowercase
df.columns = [x.lower() for x in df.columns]
df

,suburb,property_type,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,month
0,essendon,townhouse,400.0,3.0,1.0,1.0,7.09,-37.754148,144.934050,1.314543,2012,12
1,melton west,house,449.0,4.0,2.0,2.0,39.40,-37.664852,144.556370,15.035350,2015,3
2,bendigo,house,270.0,3.0,1.0,2.0,134.93,-36.738890,144.254916,102.273003,2013,6
3,southbank,Apartment / Unit / Flat,585.0,2.0,2.0,1.0,1.56,-37.827305,144.959050,1.219429,2015,4
4,rowville,house,390.0,3.0,1.0,2.0,26.46,-37.920490,145.232380,7.008687,2015,3
...,...,...,...,...,...,...,...,...,...,...,...,...
745584,brighton,house,650.0,3.0,2.0,2.0,9.70,-37.895790,145.000020,0.378541,2016,9
745585,carnegie,house,405.0,2.0,1.0,1.0,11.21,-37.889412,145.047320,0.449000,2016,11
745586,bacchus marsh,house,360.0,4.0,2.0,2.0,50.05,-37.690685,144.415370,27.792091,2016,7
745587,heathmont,house,350.0,3.0,1.0,2.0,23.53,-37.827587,145.230380,1.250880,2017,1


In [140]:
# join the two dataframes vertically
new_domaindf = new_domaindf.drop('postcode', axis=1)
df = pd.concat([df, new_domaindf], axis=0)

In [141]:
df['property_type'] = df['property_type'].str.lower()

In [142]:
df

#  this was preprocessing for the domain data and old data after they were joined together

,suburb,property_type,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,month
0,essendon,townhouse,400.0,3.0,1.0,1.0,7.090000,-37.754148,144.934050,1.314543,2012,12
1,melton west,house,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,3
2,bendigo,house,270.0,3.0,1.0,2.0,134.930000,-36.738890,144.254916,102.273003,2013,6
3,southbank,apartment / unit / flat,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,4
4,rowville,house,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,3
...,...,...,...,...,...,...,...,...,...,...,...,...
14693,burwood east,townhouse,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,9
14694,werribee south,house,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,9
14695,ferntree gully,apartment / unit / flat,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,9
14696,coburg,townhouse,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,8


## Add the school data

In [143]:
school_df = pd.read_csv('../data/curated/schools_preprocessed.csv')

In [144]:
school_df

#  joined the aggregated school data with the main data

,Unnamed: 0,suburb,num_primary,num_secondary_public,num_secondary_private,num_secondary_catholic,num_edu_centre,v_high_vcaa,high_vcaa,good_vcaa
0,0,abbotsford,2,0,1,0,0,0.0,0.0,0.0
1,1,aberfeldie,1,0,0,1,0,0.0,0.0,1.0
2,2,aintree,1,0,0,0,0,0.0,0.0,0.0
3,3,aireys inlet,1,0,0,0,0,0.0,0.0,0.0
4,4,airly,1,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
950,950,yea,2,1,0,0,0,0.0,0.0,0.0
951,951,yering,1,0,0,0,0,0.0,0.0,0.0
952,952,yinnar,1,0,0,0,0,0.0,0.0,0.0
953,953,yinnar south,1,0,0,0,0,0.0,0.0,0.0


In [145]:
school_df = school_df.drop(columns=['Unnamed: 0'])

suburbs_in_df =  list((df['suburb'].str.lower().unique()))

suburbs_in_df.sort()
print(suburbs_in_df)
# len(suburbs_in_df)

schools_in_df = list(school_df['suburb'].str.lower().unique())
schools_in_df.sort()
print(schools_in_df)

['abbotsford', 'aberfeldie', 'acheron', 'aintree', 'aireys inlet', 'airly', 'airport west', 'albanvale', 'albert park', 'alberton', 'alberton west', 'albion', 'alexandra', 'alfredton', 'allans flat', 'allansford', 'allendale', 'allestree', 'alma', 'alphington', 'altona', 'altona meadows', 'altona north', 'alvie', 'amherst', 'anakie', 'ancona', 'anglesea', 'apollo bay', 'apsley', 'ararat', 'arcadia', 'archies creek', 'ardeer', 'ardmona', 'argyle', 'armadale', 'armstrong', 'armstrong creek', 'arnold', 'arthurs creek', 'arthurs seat', 'ascot', 'ascot vale', 'ashbourne', 'ashburton', 'ashwood', 'aspendale', 'aspendale gardens', 'athlone', 'attwood', 'avalon', 'avenel', 'avoca', 'avondale heights', 'avonsleigh', 'axe creek', 'axedale', 'bacchus marsh', 'baddaginnie', 'badger creek', 'bagshot', 'bairnsdale', 'bakery hill', 'balaclava', 'bald hills', 'balintore', 'ballan', 'ballarat', 'ballarat central', 'ballarat east', 'ballarat north', 'ballendella', 'balliang', 'balmoral', 'balnarring', '

In [146]:
print(len(schools_in_df))
print(len(suburbs_in_df))
print(len(suburbs_in_df) - len(schools_in_df))

# since the number of suburbs in the main data is more than the number of suburbs in the school data, we can infer that there are some suburbs in the main data that do not have schools
#  so we made the data for those suburbs to be 0

955
1621
666


In [147]:
# use set to find the difference between the two lists
difference_in_suburbs = set(suburbs_in_df) - set(schools_in_df)

# create a list of dictionaries for the new rows
new_rows = [{'suburb': i, 'v_high_vcaa': 0, 'high_vcaa': 0, 'good_vcaa': 0, 'num_primary': 0, 'num_secondary_public': 0, 'num_secondary_private': 0, 'num_secondary_catholic': 0, 'num_edu_centre': 0} for i in difference_in_suburbs]

# concatenate the new rows to the school_df dataframe
school_df = pd.concat([school_df, pd.DataFrame(new_rows)], ignore_index=True)


In [148]:
df_with_schools = pd.merge(df, school_df, how='left', on='suburb')
#  the school data to merge with the main thing 

In [149]:
df_with_schools

,suburb,property_type,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,month,num_primary,num_secondary_public,num_secondary_private,num_secondary_catholic,num_edu_centre,v_high_vcaa,high_vcaa,good_vcaa
0,essendon,townhouse,400.0,3.0,1.0,1.0,7.090000,-37.754148,144.934050,1.314543,2012,12,5,1,1,2,0,1.0,0.0,3.0
1,melton west,house,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,3,4,0,1,0,0,0.0,0.0,1.0
2,bendigo,house,270.0,3.0,1.0,2.0,134.930000,-36.738890,144.254916,102.273003,2013,6,7,3,2,1,2,1.0,0.0,0.0
3,southbank,apartment / unit / flat,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,4,1,1,0,0,0,0.0,1.0,0.0
4,rowville,house,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,3,5,1,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760282,burwood east,townhouse,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,9,2,1,0,0,1,0.0,0.0,0.0
760283,werribee south,house,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,9,0,0,0,0,0,0.0,0.0,0.0
760284,ferntree gully,apartment / unit / flat,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,9,7,0,2,1,1,0.0,0.0,1.0
760285,coburg,townhouse,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,8,6,1,1,1,1,0.0,0.0,0.0


## House type conversion to dummy variable 

In [150]:
# convert the columns to dummy variables
# use one hot encoding to convert the columns to dummy variables

df_with_schools = pd.get_dummies(df_with_schools, columns=['property_type'])
# the one hot encoding

In [151]:
df_with_schools.isna().sum()        

suburb                                   0
rent                                     0
num_bedrooms                             0
num_bathrooms                            0
parking                                  0
distance_to_cbd                          0
latitude                                 0
longitude                                0
nearst_train_station_dist                2
year                                     0
month                                    0
num_primary                              0
num_secondary_public                     0
num_secondary_private                    0
num_secondary_catholic                   0
num_edu_centre                           0
v_high_vcaa                              0
high_vcaa                                0
good_vcaa                                0
property_type_apartment / unit / flat    0
property_type_duplex                     0
property_type_duplex-semi-detached       0
property_type_house                      0
property_ty

In [152]:
df_with_schools.dropna(inplace=True)

#  drop the schools that have no data

In [153]:
df_with_schools['suburb'] = df_with_schools['suburb'].str.replace(r'\d+', '', regex=True)
df_with_schools['suburb'] = df_with_schools['suburb'].str.strip()

#  regulate the suburb data

In [154]:
df_with_schools


,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,good_vcaa,property_type_apartment / unit / flat,property_type_duplex,property_type_duplex-semi-detached,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa
0,essendon,400.0,3.0,1.0,1.0,7.090000,-37.754148,144.934050,1.314543,2012,...,3.0,False,False,False,False,False,False,False,True,False
1,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,1.0,False,False,False,True,False,False,False,False,False
2,bendigo,270.0,3.0,1.0,2.0,134.930000,-36.738890,144.254916,102.273003,2013,...,0.0,False,False,False,True,False,False,False,False,False
3,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,0.0,True,False,False,False,False,False,False,False,False
4,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,0.0,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760282,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,0.0,False,False,False,False,False,False,False,True,False
760283,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,0.0,False,False,False,True,False,False,False,False,False
760284,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,1.0,True,False,False,False,False,False,False,False,False
760285,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,0.0,False,False,False,False,False,False,False,True,False


## Attach the LGA information 

In [155]:
LGA_to_suburb = gpd.read_file('../data/curated/lga_coord/lga_coordinates.shp', encoding='latin1')


In [156]:
df_with_schools = gpd.GeoDataFrame(df_with_schools, geometry=gpd.points_from_xy(df_with_schools.longitude, df_with_schools.latitude))

df_with_schools = df_with_schools.set_crs(epsg=4326, allow_override=True) 

df_with_schools = df_with_schools.to_crs(epsg=32755) 
#  convert the data to the same crs as the LGA data


In [157]:
LGA_to_suburb = LGA_to_suburb.to_crs(epsg=32755)

In [158]:
LGA_to_suburb

,lga,geometry
0,brimbank,"POLYGON ((302996.378 5822556.76, 302999.053 58..."
1,nillumbik,"POLYGON ((333957.676 5827142.708, 333740.86 58..."
2,cardinia,"POLYGON ((390873.597 5779414.853, 390870.013 5..."
3,moonee valley,"POLYGON ((312039.996 5817320.42, 312020.238 58..."
4,yarra ranges,"POLYGON ((394303.523 5812813.638, 394345.719 5..."
...,...,...
82,benalla,"POLYGON ((413564.139 5918299.438, 413521.822 5..."
83,murrindindi,"MULTIPOLYGON (((398344.105 5845905.652, 398141..."
84,mansfield,"POLYGON ((408678.164 5875439.111, 408671.505 5..."
85,lake mountain alpine resort (uninc),"POLYGON ((401236.025 5846375.267, 401239.593 5..."


In [159]:
df_with_schools

,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_apartment / unit / flat,property_type_duplex,property_type_duplex-semi-detached,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,geometry
0,essendon,400.0,3.0,1.0,1.0,7.090000,-37.754148,144.934050,1.314543,2012,...,False,False,False,False,False,False,False,True,False,POINT (318001.273 5819452.697)
1,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,False,False,False,True,False,False,False,False,False,POINT (284466.399 5828559.856)
2,bendigo,270.0,3.0,1.0,2.0,134.930000,-36.738890,144.254916,102.273003,2013,...,False,False,False,True,False,False,False,False,False,POINT (254893.683 5930579.005)
3,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,True,False,False,False,False,False,False,False,False,POINT (320381.145 5811383.124)
4,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,False,False,False,True,False,False,False,False,False,POINT (344634.189 5801533.596)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760282,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,False,False,False,False,False,False,False,True,False,POINT (335724.226 5808442.4)
760283,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,False,False,False,True,False,False,False,False,False,POINT (298622.95 5795770.353)
760284,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,True,False,False,False,False,False,False,False,False,POINT (347655.729 5805798.253)
760285,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,False,False,False,False,False,False,False,True,False,POINT (322231.864 5821418.894)


In [160]:
# join the two dataframes
df_lga = gpd.sjoin(df_with_schools, LGA_to_suburb, how='left', predicate='within')

#  join the dataframe so that we can get the LGA data, statistic for each rental property

In [161]:
df_lga

,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_duplex-semi-detached,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,geometry,index_right,lga
0,essendon,400.0,3.0,1.0,1.0,7.090000,-37.754148,144.934050,1.314543,2012,...,False,False,False,False,False,True,False,POINT (318001.273 5819452.697),3.0,moonee valley
1,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,False,True,False,False,False,False,False,POINT (284466.399 5828559.856),25.0,melton
2,bendigo,270.0,3.0,1.0,2.0,134.930000,-36.738890,144.254916,102.273003,2013,...,False,True,False,False,False,False,False,POINT (254893.683 5930579.005),66.0,greater bendigo
3,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,False,False,False,False,False,False,False,POINT (320381.145 5811383.124),64.0,melbourne
4,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,False,True,False,False,False,False,False,POINT (344634.189 5801533.596),9.0,knox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760282,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,False,False,False,False,False,True,False,POINT (335724.226 5808442.4),10.0,whitehorse
760283,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,False,True,False,False,False,False,False,POINT (298622.95 5795770.353),67.0,wyndham
760284,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,False,False,False,False,False,False,False,POINT (347655.729 5805798.253),9.0,knox
760285,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,False,False,False,False,False,True,False,POINT (322231.864 5821418.894),17.0,darebin


In [162]:
lga_in_df = list(df_lga['lga'].unique())

In [163]:
print(len(list(df_lga.isna().sum())))

31


In [164]:
df_lga[df_lga['lga'].isna()]
#  the na values are the ones that are not in the LGA data, and they seem to have random longitudes and latitudes
#  outside of the Victoria region

,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_duplex-semi-detached,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,geometry,index_right,lga
1639,echuca,230.0,2.0,1.0,2.0,191.08,-36.102927,144.758224,163.854768,2013,...,False,True,False,False,False,False,False,POINT (298201.059 6002308.439),NaN,NaN
1837,echuca,310.0,3.0,2.0,2.0,190.96,-36.104015,144.757446,163.732921,2013,...,False,True,False,False,False,False,False,POINT (298133.799 6002186.117),NaN,NaN
2207,echuca,300.0,4.0,2.0,1.0,191.34,-36.100731,144.756353,164.095781,2013,...,False,True,False,False,False,False,False,POINT (298026.985 6002548.182),NaN,NaN
3093,echuca,390.0,3.0,2.0,4.0,190.20,-36.111075,144.755276,162.946558,2013,...,False,False,False,False,False,True,False,POINT (297956.526 6001398.351),NaN,NaN
4430,echuca,230.0,2.0,1.0,2.0,191.08,-36.102927,144.758224,163.854768,2013,...,False,True,False,False,False,False,False,POINT (298201.059 6002308.439),NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
703998,sanctuary lakes,630.0,5.0,2.0,2.0,1335.80,-28.140937,153.481949,1308.128524,2013,...,False,True,False,False,False,False,False,POINT (1137249.611 6870137.125),NaN,NaN
705200,echuca,235.0,2.0,1.0,1.0,190.06,-36.111599,144.765434,162.904431,2014,...,False,False,False,False,False,False,False,POINT (298872.309 6001361.294),NaN,NaN
705697,sanctuary lakes,400.0,4.0,2.0,2.0,1982.04,-20.289488,148.727028,1950.250020,2013,...,False,True,False,False,False,False,False,POINT (680344.489 7755540.72),NaN,NaN
706226,willow grove,320.0,4.0,1.0,3.0,16408.75,43.116465,-77.555073,18699.673605,2013,...,False,True,False,False,False,False,False,POINT (4112045.111 24150291.888),NaN,NaN


In [165]:
df_lga.dropna(subset=['lga'], inplace=True)

In [166]:
df_lga

,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_duplex-semi-detached,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,geometry,index_right,lga
0,essendon,400.0,3.0,1.0,1.0,7.090000,-37.754148,144.934050,1.314543,2012,...,False,False,False,False,False,True,False,POINT (318001.273 5819452.697),3.0,moonee valley
1,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,False,True,False,False,False,False,False,POINT (284466.399 5828559.856),25.0,melton
2,bendigo,270.0,3.0,1.0,2.0,134.930000,-36.738890,144.254916,102.273003,2013,...,False,True,False,False,False,False,False,POINT (254893.683 5930579.005),66.0,greater bendigo
3,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,False,False,False,False,False,False,False,POINT (320381.145 5811383.124),64.0,melbourne
4,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,False,True,False,False,False,False,False,POINT (344634.189 5801533.596),9.0,knox
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760282,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,False,False,False,False,False,True,False,POINT (335724.226 5808442.4),10.0,whitehorse
760283,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,False,True,False,False,False,False,False,POINT (298622.95 5795770.353),67.0,wyndham
760284,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,False,False,False,False,False,False,False,POINT (347655.729 5805798.253),9.0,knox
760285,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,False,False,False,False,False,True,False,POINT (322231.864 5821418.894),17.0,darebin


In [167]:
df_lga['year'].value_counts()

year
2015    165712
2016    155556
2017    149800
2013    128084
2014    114235
2012     32179
2024     11997
2023       958
2022       661
2021       448
2020       184
2019       102
2018        60
2011         2
2010         2
2003         1
Name: count, dtype: int64

## Crime rate

In [168]:
crime_df = pd.read_csv('../data/curated/crime_by_lga.csv')
# the crime data

In [169]:
crime_df.drop(columns=['Unnamed: 0'], inplace=True)
lgas_crime = list(crime_df['Local Government Area'].unique())
# the LGAs in the crime data

In [170]:
set(lga_in_df) - set(lgas_crime)

{'colac otway',
 'falls creek alpine resort (uninc)',
 'mount buller alpine resort (uninc)',
 nan}

In [171]:
df_lga = df_lga[df_lga['year'] >= 2015]

In [172]:
df_lga

,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_duplex-semi-detached,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,geometry,index_right,lga
1,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,False,True,False,False,False,False,False,POINT (284466.399 5828559.856),25.0,melton
3,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,False,False,False,False,False,False,False,POINT (320381.145 5811383.124),64.0,melbourne
4,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,False,True,False,False,False,False,False,POINT (344634.189 5801533.596),9.0,knox
6,springvale,400.0,4.0,2.0,1.0,21.750000,-37.940765,145.151440,0.985106,2015,...,False,True,False,False,False,False,False,POINT (337563.967 5799145.749),22.0,greater dandenong
8,ballarat east,290.0,3.0,1.0,1.0,98.350000,-37.568535,143.889040,73.684221,2015,...,False,False,False,False,False,False,False,POINT (225238.592 5837504.612),41.0,ballarat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760282,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,False,False,False,False,False,True,False,POINT (335724.226 5808442.4),10.0,whitehorse
760283,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,False,True,False,False,False,False,False,POINT (298622.95 5795770.353),67.0,wyndham
760284,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,False,False,False,False,False,False,False,POINT (347655.729 5805798.253),9.0,knox
760285,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,False,False,False,False,False,True,False,POINT (322231.864 5821418.894),17.0,darebin


In [173]:
# left join the two dataframes
df_lga_with_crime = pd.merge(df_lga, crime_df, how='left', left_on=['lga','year'], right_on=['Local Government Area','Year'])

# merge the crime data with the main data, on the LGA and year

In [174]:
df_lga_with_crime.dropna(inplace=True)
# drop the na values

## Population column addition 

In [175]:
# read lga pop data 
lga_pop = pd.read_csv('../data/curated/lga_pop.csv')

In [176]:
list_lga   = list(df_lga_with_crime['lga'].unique())
list_lga_pop = list(lga_pop['lga'].unique())

set(list_lga) - set(list_lga_pop)

# the LGAs that are in the main data but not in the population data

set()

In [177]:
lga_pop

,lga,year,lga_population
0,alpine,2013.0,12283.00000
1,alpine,2014.0,12355.00000
2,alpine,2015.0,12429.00000
3,alpine,2016.0,12578.00000
4,alpine,2017.0,12687.00000
...,...,...,...
943,yarriambiack,2020.0,6641.00000
944,yarriambiack,2021.0,6510.00000
945,yarriambiack,2022.0,6437.00000
946,yarriambiack,2023.0,6389.00000


In [178]:
lga_pop = lga_pop[lga_pop['year'] >= 2015]
df_final = pd.merge(df_lga_with_crime, lga_pop, how='left', left_on=['lga','year'], right_on=['lga','year'])

# merge the population data with the main data

In [179]:
df_final[df_final.isna().any(axis=1)]

# the na values are the ones that are not in the population data 


,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_studio,property_type_townhouse,property_type_villa,geometry,index_right,lga,Local Government Area,Year,"Rate per 100,000 population",lga_population


In [180]:
df_final.columns

df_final.drop(columns=['geometry', 'index_right','Local Government Area'], inplace=True)
# drop the columns that are not needed
# the geometry column is not needed because it is the same as the longitude and latitude column

In [181]:
df_final


,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,lga,Year,"Rate per 100,000 population",lga_population
0,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,True,False,False,False,False,False,melton,2015.0,7518.066931,134915.000000
1,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,False,False,False,False,False,False,melbourne,2015.0,25240.370565,136872.000000
2,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,True,False,False,False,False,False,knox,2015.0,6145.557656,158700.000000
3,springvale,400.0,4.0,2.0,1.0,21.750000,-37.940765,145.151440,0.985106,2015,...,True,False,False,False,False,False,greater dandenong,2015.0,11086.049686,156061.000000
4,ballarat east,290.0,3.0,1.0,1.0,98.350000,-37.568535,143.889040,73.684221,2015,...,False,False,False,False,False,False,ballarat,2015.0,10860.375053,101479.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484553,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,False,False,False,False,True,False,whitehorse,2023.0,4950.206842,178639.000000
484554,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,True,False,False,False,False,False,wyndham,2022.0,5270.557664,309398.000000
484555,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,False,False,False,False,False,False,knox,2022.0,5691.041624,159549.000000
484556,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,False,False,False,False,True,False,darebin,2024.0,9172.670623,155788.090972


## Save the dataset 

In [182]:
df_final.to_csv('../data/curated/forcast_dataset.csv', index=False)


In [183]:
df_final
# the final dataset

,suburb,rent,num_bedrooms,num_bathrooms,parking,distance_to_cbd,latitude,longitude,nearst_train_station_dist,year,...,property_type_house,property_type_retirement living,property_type_serviced apartment,property_type_studio,property_type_townhouse,property_type_villa,lga,Year,"Rate per 100,000 population",lga_population
0,melton west,449.0,4.0,2.0,2.0,39.400000,-37.664852,144.556370,15.035350,2015,...,True,False,False,False,False,False,melton,2015.0,7518.066931,134915.000000
1,southbank,585.0,2.0,2.0,1.0,1.560000,-37.827305,144.959050,1.219429,2015,...,False,False,False,False,False,False,melbourne,2015.0,25240.370565,136872.000000
2,rowville,390.0,3.0,1.0,2.0,26.460000,-37.920490,145.232380,7.008687,2015,...,True,False,False,False,False,False,knox,2015.0,6145.557656,158700.000000
3,springvale,400.0,4.0,2.0,1.0,21.750000,-37.940765,145.151440,0.985106,2015,...,True,False,False,False,False,False,greater dandenong,2015.0,11086.049686,156061.000000
4,ballarat east,290.0,3.0,1.0,1.0,98.350000,-37.568535,143.889040,73.684221,2015,...,False,False,False,False,False,False,ballarat,2015.0,10860.375053,101479.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484553,burwood east,1050.0,4.0,3.0,2.0,15.825365,-37.856688,145.132631,2.098144,2023,...,False,False,False,False,True,False,whitehorse,2023.0,4950.206842,178639.000000
484554,werribee south,560.0,4.0,2.0,2.0,28.143669,-37.963369,144.707632,8.197645,2022,...,True,False,False,False,False,False,wyndham,2022.0,5270.557664,309398.000000
484555,ferntree gully,700.0,3.0,2.0,2.0,28.016795,-37.882581,145.267643,2.431007,2022,...,False,False,False,False,False,False,knox,2022.0,5691.041624,159549.000000
484556,coburg,835.0,3.0,3.0,2.0,8.289849,-37.737269,144.982527,1.594189,2024,...,False,False,False,False,True,False,darebin,2024.0,9172.670623,155788.090972
